# Qubix Time Dilation Simulation — Interactive Notebook

This notebook is a simplified, interactive environment for exploring a toy simulation
of “quantum collapse” with time-dilation effects. **No outputs** are pre-saved, so you
can run cells fresh, see plots inline, and experiment with parameters.

## Usage
1. Run all cells.
2. Use the interactive UI (sliders, dropdowns) to set your parameters.
3. Click **Run Simulation**.
4. Observe 2D slices, 3D Plotly visualizations, local time charts, and a single-run memory histogram.

Enjoy!

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Plotly for interactive 3D
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

import os, csv, datetime, io, sys, time, random

SHOW_PLOTS = True  # default interactive behavior
SHOW_3D = True    # toggle for 3D Plotly visualization

class SimulationConfig:
    def __init__(self, grid_size=(50, 50, 50), max_steps=100):
        self.speed_of_light = 1.0
        self.G = 1e-3
        self.M = 1e3
        self.measure_prob = 0.1
        self.max_steps = max_steps
        self.grid_size = grid_size
        self.gravity_center = tuple(s / 2 for s in self.grid_size)
        self.active_modes = []
        self.high_speed_fraction = 0.4
        self.high_speed_value = 0.7

class Qubix:
    def __init__(self, pos, config):
        self.pos = pos
        self.config = config
        self.collapsed_at = None
        self.memory = 0.0
        self.quantum_time = 0.0
        if ("gravity" in config.active_modes) or ("combined" in config.active_modes):
            dist = np.linalg.norm(np.array(pos) - np.array(config.gravity_center))
            self.measure_prob = 0.1 * np.exp(-0.05 * dist)
        else:
            self.measure_prob = config.measure_prob

    def try_collapse(self, local_time):
        # If already collapsed, increment memory a bit
        if self.collapsed_at is not None:
            self.memory = min(1.0, self.memory + 0.01)
            return False
        # Attempt new collapse
        if random.random() < self.measure_prob:
            self.collapsed_at = local_time
            self.quantum_time += local_time
            self.memory = 1.0
            return True
        # No collapse => memory increments slightly
        self.memory = min(1.0, self.memory + 0.005)
        return False

def gravitational_factor(pos, center, G, M, c):
    r = np.linalg.norm(np.array(pos) - np.array(center))
    r = max(r, 1e-5)
    return np.sqrt(max(1 - (2 * G * M)/(r * c**2), 0.0))

def velocity_factor(velocity, c):
    v = np.linalg.norm(velocity)
    v = min(v, 0.9999 * c)
    return np.sqrt(1 - v**2 / c**2)

def combined_dilation(pos, velocity, config):
    factor = 1.0
    if ("gravity" in config.active_modes) or ("combined" in config.active_modes):
        factor *= gravitational_factor(pos, config.gravity_center, config.G, config.M, config.speed_of_light)
    if ("velocity" in config.active_modes) or ("combined" in config.active_modes):
        factor *= velocity_factor(velocity, config.speed_of_light)
    return factor

class QubixGrid:
    def __init__(self, config):
        self.config = config
        self.grid = {}
        self.velocities = {}
        self.collapse_density_map = np.zeros(config.grid_size)
        self.local_time_map = np.zeros(config.grid_size)

        for x in range(config.grid_size[0]):
            for y in range(config.grid_size[1]):
                for z in range(config.grid_size[2]):
                    pos = (x, y, z)
                    self.grid[pos] = Qubix(pos, config)

                    if ("velocity" in config.active_modes) or ("combined" in config.active_modes):
                        if random.random() < config.high_speed_fraction:
                            speed = np.random.uniform(0.4, config.high_speed_value)
                            direction = np.random.normal(0,1,3)
                            direction /= (np.linalg.norm(direction)+1e-8)
                            self.velocities[pos] = direction * speed
                        else:
                            self.velocities[pos] = np.zeros(3)
                    else:
                        self.velocities[pos] = np.zeros(3)

    def step(self, step_number):
        local_times = []
        for pos, vox in self.grid.items():
            velocity = self.velocities[pos]
            dilation = combined_dilation(pos, velocity, self.config)
            local_t = step_number * dilation
            if vox.try_collapse(local_t):
                self.collapse_density_map[pos] += 1
            self.local_time_map[pos] = local_t
            local_times.append(local_t)
        return local_times

    def print_stats(self, global_time):
        memories = [v.memory for v in self.grid.values()]
        qts = [v.quantum_time for v in self.grid.values()]
        print(f"Memory: Min={min(memories):.3f}, Max={max(memories):.3f}, ",
              f"Avg={np.mean(memories):.3f}")
        print(f"Avg Δ (Quantum - Global Time) = {np.mean(qts) - global_time:.3f}")

def run_simulation(modes, config):
    grid = QubixGrid(config)
    steps = []
    local_avgs = []

    for step_num in range(1, config.max_steps+1):
        local_times = grid.step(step_num)
        steps.append(step_num)
        local_avgs.append(np.mean(local_times))

    grid.print_stats(global_time=config.max_steps)
    return steps, local_avgs, grid

def plot_clocks(steps, local_avgs, mode):
    plt.figure(figsize=(6,4))
    plt.plot(steps, local_avgs, label=f"Avg Local Time ({mode})", color='teal')
    plt.plot(steps, steps, '--', label="Global", color='gray')
    plt.xlabel("Global Steps")
    plt.ylabel("Local Time")
    plt.title("Time Dilation Chart")
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_heatmaps(grid, z_slice=9):
    # 2D slice of collapse density
    plt.figure(figsize=(5,4))
    plt.imshow(grid.collapse_density_map[:,:,z_slice].T, origin='lower', cmap='plasma')
    plt.colorbar(label="Collapse Count")
    plt.title(f"Collapse Density (z={z_slice})")
    plt.show()

    # 2D slice of local time
    plt.figure(figsize=(5,4))
    plt.imshow(grid.local_time_map[:,:,z_slice].T, origin='lower', cmap='viridis')
    plt.colorbar(label="Local Time")
    plt.title(f"Time Dilation z={z_slice}")
    plt.show()

def plot_interactive_3d(grid):
    xs, ys, zs, colors, texts = [], [], [], [], []
    for (x,y,z), vox in grid.grid.items():
        xs.append(x)
        ys.append(y)
        zs.append(z)
        colors.append(vox.memory)
        texts.append(f"({x},{y},{z}) mem={vox.memory:.3f}")

    fig = go.Figure(data=[go.Scatter3d(
        x=xs, y=ys, z=zs,
        mode='markers',
        marker=dict(
            size=4,
            color=colors,
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title="memory")
        ),
        text=texts,
        hoverinfo='text'
    )])

    fig.update_layout(
        title="Interactive 3D (Memory)",
        margin=dict(l=10, r=10, b=10, t=40),
        scene=dict(aspectmode='cube')
    )
    fig.show()


## Single-Run Memory Histogram
We'll define a function to show the final memory distribution for the *current* run.

In [2]:
def plot_memory_histogram(grid):
    """
    Plot a histogram of memory values from the current grid.
    """
    memories = [v.memory for v in grid.grid.values()]

    plt.figure(figsize=(6, 4))
    plt.hist(memories, bins=30, alpha=0.7, color='blue')
    plt.xlabel("Memory")
    plt.ylabel("Count")
    plt.title("Memory Distribution (Single Run)")
    plt.grid(True)
    plt.show()


## Interactive UI
Use the sliders and dropdown to configure a single run, then see 2D slices, 3D interactive plot, clocks, and final memory histogram.

In [3]:
mode_selector = widgets.Dropdown(
    options=["standard", "gravity", "velocity", "combined"],
    value="standard",
    description="Mode:"
)

mass_slider = widgets.FloatSlider(
    value=1000.0,
    min=100.0,
    max=5000.0,
    step=100.0,
    description="Mass (M):",
    continuous_update=False
)

g_slider = widgets.FloatSlider(
    value=0.001,
    min=0.0,
    max=0.01,
    step=0.0005,
    description="G:",
    continuous_update=False
)

collapse_slider = widgets.FloatSlider(
    value=0.1,
    min=0.01,
    max=0.5,
    step=0.01,
    description="Coll. Prob:",
    continuous_update=False
)

speed_slider = widgets.FloatSlider(
    value=0.4,
    min=0.0,
    max=1.0,
    step=0.05,
    description="Fast %:",
    continuous_update=False
)

steps_slider = widgets.IntSlider(
    value=30,
    min=10,
    max=200,
    step=10,
    description="Steps:",
    continuous_update=False
)

grid_x_slider = widgets.IntSlider(
    value=20,
    min=10,
    max=50,
    step=5,
    description="Grid X:"
)

grid_y_slider = widgets.IntSlider(
    value=20,
    min=10,
    max=50,
    step=5,
    description="Grid Y:"
)

grid_z_slider = widgets.IntSlider(
    value=20,
    min=10,
    max=50,
    step=5,
    description="Grid Z:"
)

run_button = widgets.Button(
    description="Run Simulation",
    button_style='success'
)

output = widgets.Output()

def on_run_clicked(b):
    with output:
        clear_output()
        grid_dims = (grid_x_slider.value, grid_y_slider.value, grid_z_slider.value)
        config = SimulationConfig(grid_size=grid_dims, max_steps=steps_slider.value)
        config.M = mass_slider.value
        config.G = g_slider.value
        config.measure_prob = collapse_slider.value
        config.high_speed_fraction = speed_slider.value
        config.active_modes = [mode_selector.value]

        now = datetime.datetime.now()
        display(Markdown(f"## Running Qubix Simulation — {mode_selector.value.upper()}"))
        meta = f"""
**Start Time:** {now.strftime('%Y-%m-%d %H:%M:%S')}  
**Grid Size:** {grid_dims}  
**Steps:** {steps_slider.value}  
**Mass (M):** {mass_slider.value}  
**G:** {g_slider.value}  
**c:** {config.speed_of_light}  
**Fast Voxel %:** {speed_slider.value * 100:.1f}%  
**Max Velocity:** {config.high_speed_value}  
"""
        display(Markdown(meta))

        # Run the simulation
        steps, local_avgs, qgrid = run_simulation([mode_selector.value], config)

        # 2D Heatmaps
        midz = int(grid_dims[2]/2)
        plot_heatmaps(qgrid, z_slice=midz)

        # 3D Plot (Plotly)
        if SHOW_3D:
            display(Markdown("### Interactive 3D Plot"))
            plot_interactive_3d(qgrid)

        # Local Time vs Global
        plot_clocks(steps, local_avgs, mode_selector.value)

        # Final step: Memory histogram
        display(Markdown("### Memory Histogram (Single Run)"))
        plot_memory_histogram(qgrid)

run_button.on_click(on_run_clicked)

ui = widgets.VBox([
    mode_selector,
    mass_slider,
    g_slider,
    collapse_slider,
    speed_slider,
    steps_slider,
    grid_x_slider,
    grid_y_slider,
    grid_z_slider,
    run_button,
    output
])

display(ui)